In [ ]:
import xlrd
import pandas as pd
import datetime
import numpy as np
import time
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
pd.set_option('display.max_rows', 30)
from matplotlib import pyplot as plt
import datetime
from geopy.distance import geodesic

In [ ]:
df_data = pd.read_csv('selected_trips.csv')

In [ ]:
df_data = df_data.drop('Unnamed: 0',axis=1)

In [ ]:
# buffer

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','raw_region'))

In [ ]:
buffer = np.array(df)

In [ ]:
# judge point in range or not

In [ ]:
col_name = df_data.columns.tolist()
col_name.insert(15,'Buffer')
df_data = df_data.reindex(columns=col_name)

In [ ]:
for i in range(0,len(df_data['Path'])):
    
    for j in range(0,len(df_data['Path'][i].split(';'))):
        
        point = Point(float(df_data['Path'][i].split(';')[j].split(' ')[0]),float(df_data['Path'][i].split(';')[j].split(' ')[1]))
        
        if Polygon(buffer).contains(point) == True:
            
            df_data['Buffer'][i] = 1
            
            break

In [ ]:
df_data_buffer = df_data[(df_data['Buffer'] == 1)]

In [ ]:
df_data_buffer = df_data_buffer.dropna()

In [ ]:
df_data_buffer = df_data_buffer.reset_index(drop = True)

In [ ]:
# get in and out timestamps

In [ ]:
col_name = df_data_buffer.columns.tolist()
col_name.insert(16,'Start_order')
df_data_buffer = df_data_buffer.reindex(columns=col_name)

In [ ]:
col_name = df_data_buffer.columns.tolist()
col_name.insert(17,'End_order')
df_data_buffer = df_data_buffer.reindex(columns=col_name)

In [ ]:
for i in range(0,len(df_data_buffer['Path'])):
    
    for j in range(0,len(df_data_buffer['Path'][i].split(';'))):
        
        point = Point(float(df_data_buffer['Path'][i].split(';')[j].split(' ')[0]),float(df_data_buffer['Path'][i].split(';')[j].split(' ')[1]))
        
        if Polygon(buffer).contains(point) == True:
                
            df_data_buffer['Start_order'][i] = j
            
            break

In [ ]:
for i in range(0,len(df_data_buffer['Path'])):
    
    for j in range(int(df_data_buffer['Start_order'][i]),len(df_data_buffer['Path'][i].split(';'))):
        
        point = Point(float(df_data_buffer['Path'][i].split(';')[j].split(' ')[0]),float(df_data_buffer['Path'][i].split(';')[j].split(' ')[1]))
        
        if Polygon(buffer).contains(point) == False:
            
            df_data_buffer['End_order'][i] = j-1
            
            break

In [ ]:
# determine the trips with the selected direction

In [ ]:
df_data_buffer = df_data_buffer.dropna()
df_data_buffer = df_data_buffer.reset_index(drop = True)

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','origin'))

In [ ]:
buffer = np.array(df)

In [ ]:
df_data_buffer = df_data_buffer.reset_index(drop = True)

In [ ]:
for i in range(0,len(df_data_buffer['Path'])):
        
    point = Point(float(df_data_buffer['Path'][i].split(';')[int(df_data_buffer['Start_order'][i])].split(' ')[0]),float(df_data_buffer['Path'][i].split(';')[int(df_data_buffer['Start_order'][i])].split(' ')[1]))
        
    if Polygon(buffer).contains(point) == False:

        df_data_buffer = df_data_buffer.drop(index=[i])

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','destination'))

In [ ]:
buffer = np.array(df)

In [ ]:
df_data_buffer = df_data_buffer.reset_index(drop = True)

In [ ]:
for i in range(0,len(df_data_buffer['Path'])):
        
    point = Point(float(df_data_buffer['Path'][i].split(';')[int(df_data_buffer['End_order'][i])].split(' ')[0]),float(df_data_buffer['Path'][i].split(';')[int(df_data_buffer['End_order'][i])].split(' ')[1]))
        
    if Polygon(buffer).contains(point) == False:

        df_data_buffer = df_data_buffer.drop(index=[i])

In [ ]:
df_data_buffer = df_data_buffer.reset_index(drop = True)

In [ ]:
# get travel time

In [ ]:
col_name = df_data_buffer.columns.tolist()
col_name.insert(18,'Buffer travel time')
df_data_buffer = df_data_buffer.reindex(columns=col_name)

In [ ]:
for i in range(0,df_data_buffer.shape[0]):
    
    start_order = int(df_data_buffer['Start_order'][i])
    
    end_order = int(df_data_buffer['End_order'][i])
    
    a = datetime.datetime.strptime(df_data_buffer['timestamp_path'][i].split(',')[end_order],'%Y-%m-%d %H:%M:%S')-datetime.datetime.strptime(df_data_buffer['timestamp_path'][i].split(',')[start_order],'%Y-%m-%d %H:%M:%S')
    
    df_data_buffer['Buffer travel time'][i] = a.total_seconds()

In [ ]:
# remove outlier travel time

In [ ]:
x = df_data_buffer.iloc[:,18].mean()
y = df_data_buffer.iloc[:,18].std()
row_index = df_data_buffer[(df_data_buffer[df_data_buffer.columns.values.tolist()[18]]>x+3*y)|(df_data_buffer[df_data_buffer.columns.values.tolist()[18]]<x-3*y)].index
df_data_buffer = df_data_buffer.drop(row_index,inplace=False) 

In [ ]:
df_data_buffer = df_data_buffer.reset_index(drop = True)

In [ ]:
# obtain mandatory and discretionary lane change trips

In [ ]:
col_name = df_data_buffer.columns.tolist()
col_name.insert(19,'Trip_category')
df_data_buffer = df_data_buffer.reindex(columns=col_name)

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','downstream'))

In [ ]:
buffer = np.array(df)

In [ ]:
for i in range(0,len(df_data_buffer['Path'])):
    
    total_path = len(df_data_buffer['Path'][i].split(';'))
    
    for j in range(int(df_data_buffer['End_order'][i])+1, total_path):
        
        point = Point(float(df_data_buffer['Path'][i].split(';')[j].split(' ')[0]),float(df_data_buffer['Path'][i].split(';')[j].split(' ')[1]))
        
        if Polygon(buffer).contains(point) == True:

            df_data_buffer['Trip_category'][i] = 'discretionary'
            
            break
        
        else:
            
            df_data_buffer['Trip_category'][i] = 'mandatory'

In [ ]:
df_data_buffer_d = df_data_buffer[(df_data_buffer['Trip_category'] == 'discretionary')]

In [ ]:
df_data_buffer_d = df_data_buffer_d.reset_index(drop=True)

In [ ]:
# distinguish detour travel 

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','offramp1'))

In [ ]:
buffer1 = np.array(df)

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','offramp2'))

In [ ]:
buffer2 = np.array(df)

In [ ]:
for i in range(0,len(df_data_buffer_d['Path'])):
    
    total_path = len(df_data_buffer_d['Path'][i].split(';'))
    
    for j in range(int(df_data_buffer_d['End_order'][i])+1, total_path):
        
        point = Point(float(df_data_buffer_d['Path'][i].split(';')[j].split(' ')[0]),float(df_data_buffer_d['Path'][i].split(';')[j].split(' ')[1]))
        
        if Polygon(buffer1).contains(point) == True or Polygon(buffer2).contains(point) == True:

            df_data_buffer_d['Trip_category'][i] = 'mandatory'
            
            break

In [ ]:
df_data_buffer_d = df_data_buffer_d[(df_data_buffer_d['Trip_category'] == 'discretionary')]

In [ ]:
df_data_buffer_d = df_data_buffer_d.reset_index(drop=True)

In [ ]:
# lane number acquisition

In [ ]:
idx = list(range(0, 10000001))
clm = ['VehicleID','TripID','Date','Time','Long','Lat','Lane_number']
df_trajectory_d = pd.DataFrame(None,index=idx,columns=clm)

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','lane1'))

In [ ]:
buffer_1 = np.array(df)

In [ ]:
df = pd.DataFrame(pd.read_excel('M1_box1.xlsx','lane3'))

In [ ]:
buffer_3 = np.array(df)

In [ ]:
k = 0
for j in range(0,len(df_data_buffer_d['VehicleID'])):
    for i in range(int(df_data_buffer_d['Start_order'][j]),int(df_data_buffer_d['End_order'][j]+1)):
        df_trajectory_d['VehicleID'][k] = df_data_buffer_d['VehicleID'][j].split('_')[0]
        df_trajectory_d['TripID'][k] = df_data_buffer_d['VehicleID'][j].split('_')[1]
        df_trajectory_d['Date'][k] = df_data_buffer_d['timestamp_path'][j].split(',')[i].split(' ')[0]
        df_trajectory_d['Time'][k] = df_data_buffer_d['timestamp_path'][j].split(',')[i].split(' ')[1]
        df_trajectory_d['Long'][k] = df_data_buffer_d['Path'][j].split(';')[i].split(' ')[0]
        df_trajectory_d['Lat'][k] = df_data_buffer_d['Path'][j].split(';')[i].split(' ')[1]
        
        point = Point(float(df_data_buffer_d['Path'][j].split(';')[i].split(' ')[0]),float(df_data_buffer_d['Path'][j].split(';')[i].split(' ')[1]))
        
        if Polygon(buffer_1).contains(point) == True:
            
            df_trajectory_d['Lane_number'][k] = 1
            
        elif Polygon(buffer_3).contains(point) == True:
            
            df_trajectory_d['Lane_number'][k] = 3
            
        else:
            
            df_trajectory_d['Lane_number'][k] = 2
            
        k = k + 1

In [ ]:
df_trajectory_d = df_trajectory_d.dropna()

In [ ]:
from scipy import stats

In [ ]:
# lane number correction

In [ ]:
col_name = df_trajectory_d .columns.tolist()
col_name.insert(7,'hour')
df_trajectory_d  = df_trajectory_d.reindex(columns=col_name)

In [ ]:
for i in range(0,df_trajectory_d.shape[0]):
    
    df_trajectory_d.loc[i,'hour'] = df_trajectory_d.loc[i,'Time'].split(':')[0]

In [ ]:
for i in range(0,df_trajectory_d.shape[0]):
    
    df_trajectory_d.loc[i,'TripID'] = str(df_trajectory_d.loc[i,'TripID']) + '_' + df_trajectory_d.loc[i,'hour']

In [ ]:
for i in df_trajectory_d['VehicleID'].unique():
    
    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
    
    for j in df_data_vehicle['TripID'].unique():
        
        df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]
        
        for k in range(0,df_data_trip.shape[0]):

            index_1 = df_data_trip[(df_data_trip['Time'] == df_data_trip.iloc[0,3])].index

            index_2 = df_data_trip[(df_data_trip['Time'] == df_data_trip.iloc[k,3])].index

            index_3 = df_data_trip[(df_data_trip['Time'] == df_data_trip.iloc[df_data_trip.shape[0]-1,3])].index
            
            if (index_2-index_1)[0] >= 4 and (index_3-index_2)[0] >= 4:
                
                lc_num = df_data_trip.loc[(index_2-4)[0]:(index_2+4)[0],'Lane_number'].tolist()
                
                # determine the duration of time points
                
                a = 1
                
                for b in [4, 3, 2, 1]:
                    
                    if lc_num[4] ==  lc_num[b-1]:
                    
                        a = a + 1

                        b = b - 1
                    
                    else:
                        
                        break
                    
                for c in [4, 5, 6, 7]:
                    
                    if lc_num[4] ==  lc_num[c+1]:
                    
                        a = a + 1

                        c = c + 1
                        
                    else:
                        
                        break
                        
                if a <= 4 and lc_num[b-1] == lc_num[c+1]:
                    
                    mode_num = stats.mode(lc_num)[0][0]
                    
                    df_trajectory_d.loc[index_2,'Lane_number'] = mode_num
                        
                    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
                    
                    df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]
                    

            elif (index_2-index_1)[0] < 4 and (index_3-index_2)[0] >= 4:

                lc_num = df_data_trip.loc[index_1[0]:(index_2+4)[0],'Lane_number'].tolist()

                mode_num = stats.mode(lc_num)[0][0]

                if mode_num != df_trajectory_d.loc[index_2,'Lane_number'].tolist()[0]:

                    df_trajectory_d.loc[index_2,'Lane_number'] = mode_num
                    
                    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
                    
                    df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]

            else:

                lc_num = df_data_trip.loc[(index_2-1)[0]:(index_3)[0],'Lane_number'].tolist()

                mode_num = stats.mode(lc_num)[0][0]

                if mode_num != df_trajectory_d.loc[index_2,'Lane_number'].tolist()[0]:

                    df_trajectory_d.loc[index_2,'Lane_number'] = mode_num
                    
                    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
                    
                    df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]

In [ ]:
# acquisition of driving speed for each point

In [ ]:
df_trajectory_d = df_trajectory_d.drop('hour',axis=1)

In [ ]:
col_name = df_trajectory_d.columns.tolist()
col_name.insert(7,'Speed')
df_trajectory_d = df_trajectory_d.reindex(columns=col_name)

In [ ]:
for i in df_trajectory_d['VehicleID'].unique():
    
    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
    
    for j in df_data_vehicle['TripID'].unique():
        
        df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]
        
        for k in range(1,df_data_trip.shape[0]):
            
            time2 = df_data_trip.iloc[k,2] +' '+ df_data_trip.iloc[k,3]
            
            time1 = df_data_trip.iloc[k-1,2] +' '+ df_data_trip.iloc[k-1,3]
            
            speed = geodesic((df_data_trip.iloc[k-1,5],df_data_trip.iloc[k-1,4]),(df_data_trip.iloc[k,5],df_data_trip.iloc[k,4])).km/((datetime.datetime.strptime(time2,'%Y-%m-%d %H:%M:%S')-datetime.datetime.strptime(time1,'%Y-%m-%d %H:%M:%S')).total_seconds()/3600)
            
            index_1 = df_data_trip[(df_data_trip['VehicleID'] == i) & (df_data_trip['TripID'] == j) & (df_data_trip['Date'] == df_data_trip.iloc[k,2]) & (df_data_trip['Time'] == df_data_trip.iloc[k,3])].index
            
            df_trajectory_d.loc[index_1,'Speed'] = speed

In [ ]:
# acquisition of lane changing or not for each point

In [ ]:
col_name = df_trajectory_d.columns.tolist()
col_name.insert(8,'Lane_change')
df_trajectory_d = df_trajectory_d.reindex(columns=col_name)

In [ ]:
for i in df_trajectory_d['VehicleID'].unique():
    
    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
    
    for j in df_data_vehicle['TripID'].unique():
        
        df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]
        
        for k in range(0,df_data_trip.shape[0]-1):
           
            if df_data_trip.iloc[k,6] != df_data_trip.iloc[k+1,6]:
                
                index_1 = df_data_trip[(df_data_trip['Time'] == df_data_trip.iloc[k,3])].index
           
                df_trajectory_d.loc[index_1,'Lane_change'] = 1
            
            elif df_data_trip.iloc[k,6] == df_data_trip.iloc[k+1,6]:
                
                index_1 = df_data_trip[(df_data_trip['Time'] == df_data_trip.iloc[k,3])].index
                
                df_trajectory_d.loc[index_1,'Lane_change'] = 0

In [ ]:
# acquisition of segment number for each point

In [ ]:
col_name = df_trajectory_d.columns.tolist()
col_name.insert(9,'Segment')
df_trajectory_d = df_trajectory_d.reindex(columns=col_name)

In [ ]:
for i in range(1,41):

    buffer = pd.DataFrame(pd.read_excel('M1_box1.xlsx','segment'+str(i)))

    buffer = np.array(buffer)

    for k in range(0,df_trajectory_d.shape[0]):

        point = Point(float(df_trajectory_d.iloc[k,4]),float(df_trajectory_d.iloc[k,5]))

        if Polygon(buffer).contains(point) == True:

            df_trajectory_d.iloc[k,9] = i

In [ ]:
# acquisition of travel time and lane changing frequency for each trip

In [ ]:
lanechanging_number = {}

lanechanging_number['number'] = {}

lanechanging_number['time'] = {}

for i in df_trajectory_d['VehicleID'].unique():
    
    df_data_vehicle = df_trajectory_d[(df_trajectory_d['VehicleID'] == i)]
    
    for j in df_data_vehicle['TripID'].unique():
        
        lanechanging_number['number'][str(i)+'_'+str(j)] = []
        
        lanechanging_number['time'][str(i)+'_'+str(j)] = []
        
        df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]
        
        for k in range(1,len(df_data_trip['Lane_number'].to_list())):
            
            if df_data_trip['Lane_number'].to_list()[k] != df_data_trip['Lane_number'].to_list()[k-1]:
                
                lanechanging_number['number'][str(i)+'_'+str(j)].append(1)
                
        lanechanging_number['number'][str(i)+'_'+str(j)] = sum(lanechanging_number['number'][str(i)+'_'+str(j)])
        
        time2 = df_data_trip.iloc[df_data_trip.shape[0]-1,2] +' '+ df_data_trip.iloc[df_data_trip.shape[0]-1,3]
            
        time1 = df_data_trip.iloc[0,2] +' '+ df_data_trip.iloc[0,3]
        
        lanechanging_number['time'][str(i)+'_'+str(j)] = (datetime.datetime.strptime(time2,'%Y-%m-%d %H:%M:%S')-datetime.datetime.strptime(time1,'%Y-%m-%d %H:%M:%S')).total_seconds()

In [ ]:
df_lane_time_d = pd.DataFrame(lanechanging_number)

In [ ]:
x = df_lane_time_d.iloc[:,1].mean()
y = df_lane_time_d.iloc[:,1].std()
row_index = df_lane_time_d[(df_lane_time_d[df_lane_time_d.columns.values.tolist()[1]]>x+3*y)|(df_lane_time_d[df_lane_time_d.columns.values.tolist()[1]]<x-3*y)].index
df_lane_time_d1 = df_lane_time_d.drop(row_index,inplace=False) 

In [ ]:
df_lane_time_d1 = df_lane_time_d1.reset_index(drop = True)

In [ ]:
# individual perspective analysis

In [ ]:
df_trajectory_d = df_trajectory_d.dropna()

In [ ]:
df_trajectory_d = df_trajectory_d.reset_index(drop=True)

In [ ]:
df_data = df_trajectory_d

In [ ]:
up_stream = {}
merging_area = {}
overtaking_area = {}
down_stream = {}

up_stream['speed'] = {}
merging_area['speed'] = {}
overtaking_area['speed'] = {}
down_stream['speed'] = {}

In [ ]:
for i in np.arange(5,105,5):
    
    up_stream['speed'][i] = []
    merging_area['speed'][i] = []
    overtaking_area['speed'][i] = []
    down_stream['speed'][i] = []

In [ ]:
for i in df_data['VehicleID'].unique():

    df_data_vehicle = df_data[(df_data['VehicleID'] == i)]

    for j in df_data_vehicle['TripID'].unique():

        df_data_trip = df_data_vehicle[(df_data_vehicle['TripID'] == j)]
        
        df_data_trip =  df_data_trip.reset_index(drop=True)
        
        row_index = df_data_trip[(df_data_trip.Lane_change ==1)].index
        
        if len(row_index) != 0:
            
            for k in range(0,len(row_index)):
                
                if row_index[k] != df_data_trip.shape[0]-1:
                
                    speed_before = df_data_trip.Speed[row_index[k]]
                    
                    speed_after = df_data_trip.Speed[row_index[k]+1]
                    
                    speed_change = speed_after - speed_before
                    
                    speed_index = int(speed_before/5)*5 + 5
                    
                    seg_index = df_data_trip.Segment[row_index[k]]
                    
                    if speed_index <= 100 :
                    
                        if 0 < seg_index and seg_index < 9:

                            up_stream['speed'][speed_index].append(speed_change)

                        elif 8 < seg_index and seg_index < 11:

                            overtaking_area['speed'][speed_index].append(speed_change)

                        elif 12 < seg_index and seg_index < 15:

                            merging_area['speed'][speed_index].append(speed_change)

                        elif 14 < seg_index:

                            down_stream['speed'][speed_index].append(speed_change)

In [ ]:
# get the speed change

In [ ]:
for i in up_stream['speed'].keys():
    
    up_stream['speed'][i] = sum(up_stream['speed'][i])/len(up_stream['speed'][i])
    
for i in up_stream['speed'].keys():
    
    down_stream['speed'][i] = sum(down_stream['speed'][i])/len(down_stream['speed'][i])
    
for i in up_stream['speed'].keys():
    
    overtaking_area['speed'][i] = sum(overtaking_area['speed'][i])/len(overtaking_area['speed'][i])
    
for i in up_stream['speed'].keys():
    
    merging_area['speed'][i] = sum(merging_area['speed'][i])/len(merging_area['speed'][i])

In [ ]:
# get the number of trips

In [ ]:
for i in up_stream['speed'].keys():
    
    up_stream['speed'][i] = len(up_stream['speed'][i])
    
for i in up_stream['speed'].keys():
    
    down_stream['speed'][i] = len(down_stream['speed'][i])
    
for i in up_stream['speed'].keys():
    
    overtaking_area['speed'][i] = len(overtaking_area['speed'][i])
    
for i in up_stream['speed'].keys():
    
    merging_area['speed'][i] = len(merging_area['speed'][i])

In [ ]:
# system perspective analysis

In [ ]:
# speed space-time

In [ ]:
df = df_data

In [ ]:
speed = {}

for i in range(0,288):
    
    speed[i] = {}
    
    for j in range(1,41):
        
        speed[i][j] = []

In [ ]:
for k in range(0,df.shape[0]):

    time = df_data.iloc[k,3]

    time_num = int(time.split(':')[0])*12 + int(int(time.split(':')[1])/5)
    
    seg_num = df_data.iloc[k,9]

    speed[time_num][seg_num].append(df_data.iloc[k,7])

In [ ]:
for i in range(0,288):
    
    for j in range(1,41):
        
        if len(speed[i][j]) != 0:

            speed[i][j] = sum(speed[i][j])/len(speed[i][j])

In [ ]:
# lane changing space-time

In [ ]:
lanechange = {}

for i in range(0,288):
    
    lanechange[i] = {}
    
    for j in range(1,41):
        
        lanechange[i][j] = []

In [ ]:
num_count = {}

for i in range(0,288):
    
    num_count[i] = {}
    
    for j in range(1,41):
        
        num_count[i][j] = []

In [ ]:
for k in range(0,df.shape[0]):

    time = df_data.iloc[k,3]

    time_num = int(time.split(':')[0])*12 + int(int(time.split(':')[1])/5)
    
    seg_num = df_data.iloc[k,9]

    lanechange[time_num][seg_num].append(df_data.iloc[k,8])
    
    num_count[time_num][seg_num].append(str(df_data.iloc[k,0])+str(df_data.iloc[k,1])+str(df_data.iloc[k,9]))

In [ ]:
for i in range(0,288):
    
    for j in range(1,41):
        
        if len(num_count[i][j]) != 0:

            num_count[i][j] = set(num_count[i][j])

In [ ]:
for i in range(0,288):
    
    for j in range(1,41):
        
        if len(lanechange[i][j]) != 0:

            lanechange[i][j] = sum(lanechange[i][j])/len(num_count[i][j])
            
        else:
            
            lanechange[i][j] = 0